In [ ]:
subcases_pattern="_0774" 
reference_location="../.." 
source_name="Crab"
osa_version='OSA10.2'
nscw=3
mosaic=0
chi2_limit=1.2
systematic_fraction=0.01

In [ ]:
if reference_location == "None": # shortcut of C.
    reference_location = "https://www.isdc.unige.ch/~ferrigno/Downloads/subcases_herX1_spi.tgz"
    
if reference_location.startswith("http://") or reference_location.startswith("https://"):
    import tarfile
    import requests
    import io
    import gzip
    
    f = gzip.open(io.BytesIO(requests.get(reference_location).content), "rb")
    
    
    with tarfile.TarFile(fileobj=f,  encoding="UTF-8") as tf:
        print(tf.getmembers())
        tf.extractall()
        
    reference_location='.'

#  ISGRI verification with SPI reference 

simultaneous observations allow easy comparisong

one should take care to have the same effective selection

normalization need to be fitted and reported

In [ ]:
%matplotlib notebook
#import matplotlib.pylab as plt


import importlib
from astroquery.simbad import Simbad
from astropy import coordinates as coord
import os
import numpy as np
import pandas as pd
import glob


In [ ]:

subcase=None

reference_spectrum = None
reference_rmf = None
scwlist=None

subcases_tried = []

for subcase_dir in glob.glob(reference_location+"/subcases/*"+subcases_pattern+"*"):
    print("found subcase in reference location dir:", subcase_dir)
    
    fns = [os.path.basename(fn) for fn in glob.glob(subcase_dir+"/*")]
    
    try:
        reference_spectrum = [fn for fn in fns if fn.startswith("spectrum")][0]
        scwlist = open(subcase_dir+"/ISGRI_scw_list.txt").read().split()       
        
        reference_rmf = sorted([fn for fn in fns if fn.startswith("rmf_")])[-1]
    except Exception as e: 
        print("failed to find spi rmf/spec have",fns, "exception:", e)
        
        subcases_tried.append(f"{subcase_dir}: {e}, files {fns}")
        
        continue
    
    if os.path.exists(subcase_dir+"/ISGRI_sources.txt"):
        ref_sources = pd.read_csv(subcase_dir+"/ISGRI_sources.txt", delimiter=",")
        print("read reference sources:", ref_sources)
    else:
        print("no reference sources, will use empty")
        ref_sources = []
            
    reference_dir=subcase_dir
    
       
    break

if scwlist is None:    
    print("unable to find spi: will NOT try without")
    raise Exception(f"subcase is not complete: no SPI or ISGRI data found, tried: {'; '.join(subcases_tried)}")

    
print("found",reference_dir, reference_spectrum, len(scwlist),len(ref_sources), ref_sources)

    
reference_spectrum, reference_rmf, scwlist

In [ ]:
ref_sources

In [ ]:
result_table = Simbad.query_object(source_name)

if result_table is None:
    print(f"\033[31mdid not find any Simbad results for {source_name}\033[0m")
    raise RuntimeError(f"\033[31mdid not find any Simbad results for {source_name}\033[0m")

source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))


result_table

In [ ]:
import random
if nscw >0:
    random.seed(0)
    cleaned_list = [
                s+"."+"001"
                for s in list(sorted(set( scwlist  ))) 
                if s.endswith("0010")
            ]
    if len(cleaned_list) > nscw:
        scw_pick = random.sample(cleaned_list,nscw)
    else:
        print("nscw (%d) > than available scw (%d), using them all"%(nscw,len(cleaned_list)))
        scw_pick = cleaned_list


    scw_list_str = ",".join(scw_pick)
else:
    scw_list_str=",".join([s+"."+"001" for s in sorted(set( scwlist  ))])

print(len(scw_list_str))

scw_list_str

In [ ]:
import oda_api.api
import importlib
importlib.reload(oda_api)

# osa versions with '-' use ic root version, only available on staging-1-3
print('will find appropriate API for OSA version', osa_version)
if '-' in osa_version:
    print('osa version has subversion - will use staging-1-3')
    disp = oda_api.api.DispatcherAPI(host="http://in.internal.odahub.io/staging-1-3/dispatcher")
else:
    try:
        disp = oda_api.api.DispatcherAPI(host="http://cdcihn.isdc.unige.ch/staging-1.2/dispatcher")
        disp.get_instrument_description("isgri")    
    except Exception as e:
        print('\033[31mFAILED to access staging-1-2, will proceed to query production: note that it can not access private data!\033[0m')
        print('\033[31mexception was: ', e,'\033[0m')
        try:
            disp = oda_api.api.DispatcherAPI(host='https://www.astro.unige.ch/cdci/astrooda/dispatch-data')
            disp.get_instrument_description("isgri")
        except:
            raise ConnectionError




In [ ]:
image = disp.get_product(instrument="isgri", 
                 product="isgri_image", 
                 product_type="Real", 
                 osa_version=osa_version,
                 E1_keV=25.0,
                 E2_keV=80.0,
                 scw_list=scw_list_str)


In [ ]:
from astropy import table
sources=image.dispatcher_catalog_1.table[image.dispatcher_catalog_1.table['significance']>=6.0]
#source = sources[sources['src_names']==source_name]
unique_sources=table.unique(sources, keys=['src_names'])

unique_sources

In [ ]:
##Removes new sources and adds our if not found
FLAG=0
torm=[]
for ID,n in enumerate(unique_sources['src_names']):
    if(n[0:3]=='NEW'):
        torm.append(ID)
    if(n==source_name):
        FLAG=1

unique_sources.remove_rows(torm)
nrows=len(unique_sources['src_names'])

if FLAG==0:
    unique_sources.add_row((0,source_name,0,source_coord.ra.deg,source_coord.dec.deg,0,2,0,0))

image.dispatcher_catalog_1.table = unique_sources

api_cat=image.dispatcher_catalog_1.get_api_dictionary()

api_cat

In [ ]:
spectrum = disp.get_product(instrument="isgri", 
                 product="isgri_spectrum", 
                 product_type="Real", 
                 osa_version=osa_version,
                 E1_keV=25.0,
                 E2_keV=80.0,
                 scw_list=scw_list_str,
                 selected_cat=api_cat)

In [ ]:
for l in spectrum._p_list:
    print(l.meta_data)

In [ ]:
specprod=[l for l in spectrum._p_list if l.meta_data['src_name'] == source_name]    

spec_fn="/tmp/isgri_spectrum_{}.fits".format(source_name.replace(' ', '_').replace('+','p'))
arf_fn="/tmp/isgri_arf_{}.fits".format(source_name.replace(' ', '_').replace('+','p'))
rmf_fn="/tmp/isgri_rmf_{}.fits".format(source_name.replace(' ', '_').replace('+','p'))

specprod[0].write_fits_file(spec_fn)
specprod[1].write_fits_file(arf_fn)
specprod[2].write_fits_file(rmf_fn)

In [ ]:

from IPython.display import Image
from IPython.display import display 
import xspec
import shutil



In [ ]:
from astropy.io import fits

if reference_spectrum is not None:
    f=fits.open(reference_dir+"/"+reference_spectrum)

    try:
        f[2].header['RESPFILE'] = 'NONE'
    except:
        f=fits.HDUList([f[0],fits.ImageHDU(),f[1]])
        f[2].header['RESPFILE'] = 'NONE'

    #f[2].header['RESPFILE']

    f.writeto("reference_spec.fits", overwrite=True)

    f=fits.open(reference_dir+"/"+reference_rmf)
    f.writeto("reference_rmf.fits", overwrite=True)
    
    reference_spec="reference_spec.fits"
    reference_rmf="reference_rmf.fits"
else:
    reference_spec="NONE"
    reference_rmf="NONE"

    

In [ ]:
reference_spec=reference_spec
reference_rmf=reference_rmf
isgri_spec=spec_fn
isgri_arf=arf_fn
isgri_rmf=rmf_fn